In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

import os
import codecs
import subprocess
from pprint import pprint
from subprocess import Popen, PIPE, STDOUT

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [2]:
import numpy
import codecs
import tempfile
import cPickle
import os
import copy
from collections import OrderedDict
import itertools

from fuel.datasets import H5PYDataset
from picklable_itertools import iter_, chain
from fuel.datasets import Dataset
from fuel.datasets import TextFile
from fuel.schemes import ConstantScheme
from fuel.streams import DataStream
from fuel.transformers import (
    Merge, Batch, Filter, Padding, SortMapping, Unpack, Mapping)
from fuel.transformers import Transformer

import os
import shutil
from collections import Counter
from theano import tensor
from toolz import merge
import numpy
import pickle
from subprocess import Popen, PIPE
import codecs

from blocks.algorithms import (GradientDescent, StepClipping,
                               CompositeRule, Adam, AdaDelta)
from blocks.extensions import FinishAfter, Printing, Timing
from blocks.extensions.monitoring import TrainingDataMonitoring
from blocks.filter import VariableFilter
from blocks.graph import ComputationGraph, apply_noise, apply_dropout
from blocks.initialization import IsotropicGaussian, Orthogonal, Constant
from blocks.main_loop import MainLoop
from blocks.model import Model
from blocks.select import Selector
from blocks.search import BeamSearch
from blocks_extras.extensions.plot import Plot

from machine_translation.checkpoint import CheckpointNMT, LoadNMT
from machine_translation.model import BidirectionalEncoder, Decoder
from machine_translation.sampling import BleuValidator, Sampler, SamplingBase
from machine_translation.stream import (get_tr_stream, get_dev_stream,
                                        _ensure_special_tokens, MTSampleStreamTransformer,
                                        get_textfile_stream, _too_long, _length, PaddingWithEOS,
                                        _oov_to_unk)
from machine_translation.evaluation import sentence_level_bleu
from machine_translation.models import MinimumRiskSequenceDecoder


from nnqe.dataset.preprocess import whitespace_tokenize

DEBUG:root:test


In [3]:
# build the training and sampling graphs for minimum risk training
# Intialize the MTSampleStreamTransformer with the sampling function

# load a model that's already trained, and start tuning it with minimum-risk
# mock-up training using the blocks main loop

In [4]:
#     def cost(self, application_call, representation, source_sentence_mask,
#              target_samples, target_samples_mask, **kwargs):



In [5]:
# TODO: change the way the models are called from the command line to allow more flexibility
# TODO: in the way they are trained

In [6]:
# create the graph which can sample from our model 
# Note that we must sample instead of getting the 1-best or N-best, because we need the randomness to make the expected
# BLEU score make sense

exp_config = {
    'src_vocab_size': 20000,
    'trg_vocab_size': 20000,
    'enc_embed': 300,
    'dec_embed': 300,
    'enc_nhids': 800,
    'dec_nhids': 800,
    'saved_parameters': '/home/chris/projects/neural_mt/archived_models/BERTHA-TEST_Adam_wmt-multimodal_internal_data_dropout'+\
    '0.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/best_bleu_model_1455464992_BLEU31.61.npz',
    'src_vocab': '/home/chris/projects/neural_mt/archived_models/BERTHA-TEST_Adam_wmt-multimodal_internal_data_dropout0'+\
    '.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/vocab.en-de.en.pkl',
    'trg_vocab': '/home/chris/projects/neural_mt/archived_models/BERTHA-TEST_Adam_wmt-multimodal_internal_data_dropout0'+\
    '.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/vocab.en-de.de.pkl',
    'src_data': '/home/chris/projects/neural_mt/archived_models/BERTHA-TEST_Adam_wmt-multimodal_internal_data_dropout'+\
    '0.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/training_data/train.en.tok.shuf',
    'trg_data': '/home/chris/projects/neural_mt/archived_models/BERTHA-TEST_Adam_wmt-multimodal_internal_data_dropout'+\
    '0.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/training_data/train.de.tok.shuf',
    'unk_id':1,
    # Bleu script that will be used (moses multi-perl in this case)
    'bleu_script': '/home/chris/projects/neural_mt/test_data/sample_experiment/tiny_demo_dataset/multi-bleu.perl',
    # Optimization related ----------------------------------------------------
    # Batch size
    'batch_size': 2,
    # This many batches will be read ahead and sorted
    'sort_k_batches': 2,
    # Optimization step rule
    'step_rule': 'AdaDelta',
    # Gradient clipping threshold
    'step_clipping': 1.,
    # Std of weight initialization
    'weight_scale': 0.01,
    'seq_len': 40,
    'finish_after': 100
}

def get_sampling_model_and_input(exp_config):
    # Create Theano variables
    encoder = BidirectionalEncoder(
        exp_config['src_vocab_size'], exp_config['enc_embed'], exp_config['enc_nhids'])

    decoder = Decoder(
        exp_config['trg_vocab_size'], exp_config['dec_embed'], exp_config['dec_nhids'],
        exp_config['enc_nhids'] * 2)

    # Create Theano variables
    logger.info('Creating theano variables')
    sampling_input = tensor.lmatrix('source')

    # Get beam search
    logger.info("Building sampling model")
    sampling_representation = encoder.apply(
        sampling_input, tensor.ones(sampling_input.shape))
    generated = decoder.generate(sampling_input, sampling_representation)

#     _, samples = VariableFilter(
#         bricks=[decoder.sequence_generator], name="outputs")(
#                  ComputationGraph(generated[1]))  # generated[1] is next_outputs
#     beam_search = BeamSearch(samples=samples)

    # build the model that will let us get a theano function from the sampling graph
    logger.info("Creating Sampling Model...")
    sampling_model = Model(generated)

    # Set the parameters from a trained models
    logger.info("Loading parameters from model: {}".format(exp_config['saved_parameters']))
    # load the parameter values from an .npz file
    param_values = LoadNMT.load_parameter_values(exp_config['saved_parameters'])
    LoadNMT.set_model_parameters(sampling_model, param_values)
    
    return sampling_model, sampling_input

test_model, theano_sampling_input = get_sampling_model_and_input(exp_config)

INFO:root:Creating theano variables
INFO:root:Building sampling model
INFO:root:Creating Sampling Model...
INFO:root:Loading parameters from model: /home/chris/projects/neural_mt/archived_models/BERTHA-TEST_Adam_wmt-multimodal_internal_data_dropout0.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/best_bleu_model_1455464992_BLEU31.61.npz
INFO:machine_translation.checkpoint: Loaded to CG (800,)         : /bidirectionalencoder/bidirectionalwmt15/backward.initial_state
INFO:machine_translation.checkpoint: Loaded to CG (1600,)        : /bidirectionalencoder/back_fork/fork_gate_inputs.b
INFO:machine_translation.checkpoint: Loaded to CG (300, 1600)    : /bidirectionalencoder/back_fork/fork_gate_inputs.W
INFO:machine_translation.checkpoint: Loaded to CG (800,)         : /bidirectionalencoder/back_fork/fork_inputs.b
INFO:machine_translation.checkpoint: Loaded to CG (300, 800)     : /bidirectionalencoder/back_fork/fork_inputs.W
INFO:machine_translation.checkpoint: Loaded to C

In [7]:
# test that we can pull samples from the model
test_sampling_func = test_model.get_theano_function()
trg_vocab = cPickle.load(open(exp_config['trg_vocab']))
trg_vocab_size = exp_config['trg_vocab_size'] - 1

trg_vocab = _ensure_special_tokens(trg_vocab, bos_idx=0,
                                   eos_idx=trg_vocab_size, unk_idx=exp_config['unk_id'])

# close over the sampling func and the trg_vocab to standardize the interface
# TODO: actually this should be a callable class with params (sampling_func, trg_vocab)
# TODO: we may be able to make this function faster by passing multiple sources for sampling at the same damn time
def sampling_func(source_seq, num_samples=1):
    print('sampling_func')
    print('num_samples: {}'.format(num_samples))
    
    def _get_true_length(seq, trg_vocab):
        try:
            return seq.tolist().index(trg_vocab['</S>']) + 1
        except ValueError:
            return len(seq)
    
    samples = []
    for _ in range(num_samples):
        # outputs of self.sampling_fn = outputs of sequence_generator.generate: next_states + [next_outputs] +
        #                 list(next_glimpses.values()) + [next_costs])
        _1, outputs, _2, _3, costs = test_sampling_func(source_seq[None, :])
        # if we are generating a single sample, the length of the output will be len(source_seq)*2
        # see decoder.generate
        # the output is a [seq_len, 1] array
        outputs = outputs.reshape(outputs.shape[0])
        outputs = outputs[:_get_true_length(outputs, trg_vocab)]
        
        samples.append(outputs)
    
    return samples

In [8]:
src_stream = get_textfile_stream(source_file=exp_config['src_data'], src_vocab=exp_config['src_vocab'],
                                         src_vocab_size=exp_config['src_vocab_size'])

# test_source_stream.sources = ('sources',)
trg_stream = get_textfile_stream(source_file=exp_config['trg_data'], src_vocab=exp_config['trg_vocab'],
                                         src_vocab_size=exp_config['trg_vocab_size'])

# Merge them to get a source, target pair
training_stream = Merge([src_stream,
                         trg_stream],
                         ('source', 'target'))

In [9]:
# sampling_transformer = MTSampleStreamTransformer(sampling_func, fake_score, num_samples=5)
sampling_transformer = MTSampleStreamTransformer(sampling_func, sentence_level_bleu, num_samples=5)

training_stream = Mapping(training_stream, sampling_transformer, add_sources=('samples', 'scores'))

In [10]:
class FlattenSamples(Transformer):
    """Adds padding to variable-length sequences.

    When your batches consist of variable-length sequences, use this class
    to equalize lengths by adding zero-padding. To distinguish between
    data and padding masks can be produced. For each data source that is
    masked, a new source will be added. This source will have the name of
    the original source with the suffix ``_mask`` (e.g. ``features_mask``).

    Elements of incoming batches will be treated as numpy arrays (i.e.
    using `numpy.asarray`). If they have more than one dimension,
    all dimensions except length, that is the first one, must be equal.

    Parameters
    ----------
    data_stream : :class:`AbstractDataStream` instance
        The data stream to wrap
    mask_sources : tuple of strings, optional
        The sources for which we need to add a mask. If not provided, a
        mask will be created for all data sources
    mask_dtype: str, optional
        data type of masks. If not provided, floatX from config will
        be used.

    """
    def __init__(self, data_stream, mask_sources=None, mask_dtype=None,
                 **kwargs):
        if data_stream.produces_examples:
            raise ValueError('the wrapped data stream must produce batches of '
                             'examples, not examples')
        super(FlattenSamples, self).__init__(
            data_stream, produces_examples=False, **kwargs)
        if mask_sources is None:
            mask_sources = self.data_stream.sources
        self.mask_sources = mask_sources
#         if mask_dtype is None:
#             self.mask_dtype = config.floatX
#         else:
#             self.mask_dtype = mask_dtype

    @property
    def sources(self):
        return self.data_stream.sources
#         sources = []
#         for source in self.data_stream.sources:
#             sources.append(source)
#             if source in self.mask_sources:
#                 sources.append(source + '_mask')
#         return tuple(sources)

    def transform_batch(self, batch):
        batch_with_flattened_samples = []
        for i, (source, source_batch) in enumerate(
                zip(self.data_stream.sources, batch)):
#             if source not in self.mask_sources:
#                 batch_with_masks.append(source_batch)
#                 continue
            if source == 'samples':
                flattened_samples = []
                for ins in source_batch:
                    for sample in ins:
                        flattened_samples.append(sample)
                batch_with_flattened_samples.append(flattened_samples)
            else:
                batch_with_flattened_samples.append(source_batch)
                        
#             shapes = [numpy.asarray(sample).shape for sample in source_batch]
#             lengths = [shape[0] for shape in shapes]
#             max_sequence_length = max(lengths)
#             rest_shape = shapes[0][1:]
#             if not all([shape[1:] == rest_shape for shape in shapes]):
#                 raise ValueError("All dimensions except length must be equal")
#             dtype = numpy.asarray(source_batch[0]).dtype

#             padded_batch = numpy.zeros(
#                 (len(source_batch), max_sequence_length) + rest_shape,
#                 dtype=dtype)
#             for i, sample in enumerate(source_batch):
#                 padded_batch[i, :len(sample)] = sample
#             batch_with_masks.append(padded_batch)

#             mask = numpy.zeros((len(source_batch), max_sequence_length),
#                                self.mask_dtype)
#             for i, sequence_length in enumerate(lengths):
#                 mask[i, :sequence_length] = 1
        return tuple(batch_with_flattened_samples)

In [11]:
# TODO: now call the main() function with the train and dev streams 
# TODO: test that the naive cost function works

 # Filter sequences that are too long
# training_stream = Filter(training_stream,
#                 predicate=_too_long(seq_len=exp_config['seq_len']))

# Replace out of vocabulary tokens with unk token
# training_stream = Mapping(training_stream,
#                  _oov_to_unk(src_vocab_size=exp_config['src_vocab_size'],
#                              trg_vocab_size=exp_config['trg_vocab_size'],
#                              unk_id=exp_config['unk_id']))

# Build a batched version of stream to read k batches ahead
training_stream = Batch(training_stream,
               iteration_scheme=ConstantScheme(
                   exp_config['batch_size']*exp_config['sort_k_batches']))

# Sort all samples in the read-ahead batch
training_stream = Mapping(training_stream, SortMapping(_length))

# Convert it into a stream again
training_stream = Unpack(training_stream)

# Construct batches from the stream with specified batch size
training_stream = Batch(
    training_stream, iteration_scheme=ConstantScheme(exp_config['batch_size']))

# Pad sequences that are short
# TODO: padding needs to be done over the flattened samples, or the Padding implementation needs to be
# modified --
# IDEA: add a transformer which flattens the target samples before we add the mask
flat_sample_stream = FlattenSamples(training_stream)

# TODO: some sources can be excluded from the padding Op
masked_stream = PaddingWithEOS(
    flat_sample_stream, [exp_config['src_vocab_size'] - 1, exp_config['trg_vocab_size'] - 1])

In [ ]:
def main(config, tr_stream, dev_stream, use_bokeh=False):

    # Create Theano variables
    logger.info('Creating theano variables')
    source_sentence = tensor.lmatrix('source')
    source_sentence_mask = tensor.matrix('source_mask')
    
#     target_samples = tensor.tensor3('samples').astype('int64')
#     target_samples_mask = tensor.tensor3('target_samples_mask').astype('int64')
    samples = tensor.lmatrix('samples')
    samples_mask = tensor.matrix('samples_mask')
    
    # scores is (batch, samples)
    scores = tensor.matrix('scores')
    # We don't need a scores mask because there should be the same number of scores for each instance
    # num samples is a hyperparameter of the model
    
#     samples_mask = tensor.matrix('s_mask')
    
    sampling_input = tensor.lmatrix('input')

    # Construct model
    logger.info('Building RNN encoder-decoder')
    encoder = BidirectionalEncoder(
        config['src_vocab_size'], config['enc_embed'], config['enc_nhids'])
#     decoder = MinimumRiskSequenceDecoder(
#         config['trg_vocab_size'], config['dec_embed'], config['dec_nhids'],
#         config['enc_nhids'] * 2)
    decoder = Decoder(
        config['trg_vocab_size'], config['dec_embed'], config['dec_nhids'],
        config['enc_nhids'] * 2)
    
    # the name is important to make sure pre-trained params get loaded correctly
#     decoder.name = 'decoder'
    
    # This is the part that is different for the MinimumRiskSequenceGenerator
    # WORKING: do we need to modify the sequence_generator directly??
    cost = decoder.expected_cost(
        encoder.apply(source_sentence, source_sentence_mask),
        source_sentence_mask, samples, samples_mask, scores)
#     cost = decoder.cost(
#         encoder.apply(source_sentence, source_sentence_mask),
#         source_sentence_mask, target_samples, target_samples_mask)

    logger.info('Creating computational graph')
    cg = ComputationGraph(cost)
    
    # Initialize model
#     logger.info('Initializing model')
#     encoder.weights_init = decoder.weights_init = IsotropicGaussian(
#         config['weight_scale'])
#     encoder.biases_init = decoder.biases_init = Constant(0)
#     encoder.push_initialization_config()
#     decoder.push_initialization_config()
#     encoder.bidir.prototype.weights_init = Orthogonal()
#     decoder.transition.weights_init = Orthogonal()
#     encoder.initialize()
#     decoder.initialize()

    # WORKING: set weights for the decoder model
    
    
    # apply dropout for regularization
#     if config['dropout'] < 1.0:
        # dropout is applied to the output of maxout in ghog
        # this is the probability of dropping out, so you probably want to make it <=0.5
#         logger.info('Applying dropout')
#         dropout_inputs = [x for x in cg.intermediary_variables
#                           if x.name == 'maxout_apply_output']
#         cg = apply_dropout(cg, dropout_inputs, config['dropout'])

    # Apply weight noise for regularization
#     if config['weight_noise_ff'] > 0.0:
#         logger.info('Applying weight noise to ff layers')
#         enc_params = Selector(encoder.lookup).get_parameters().values()
#         enc_params += Selector(encoder.fwd_fork).get_parameters().values()
#         enc_params += Selector(encoder.back_fork).get_parameters().values()
#         dec_params = Selector(
#             decoder.sequence_generator.readout).get_parameters().values()
#         dec_params += Selector(
#             decoder.sequence_generator.fork).get_parameters().values()
#         dec_params += Selector(decoder.transition.initial_transformer).get_parameters().values()
#         cg = apply_noise(cg, enc_params+dec_params, config['weight_noise_ff'])

    # TODO: weight noise for recurrent params isn't currently implemented -- see config['weight_noise_rec']
    # Print shapes
#     shapes = [param.get_value().shape for param in cg.parameters]
#     logger.info("Parameter shapes: ")
#     for shape, count in Counter(shapes).most_common():
#         logger.info('    {:15}: {}'.format(shape, count))
#     logger.info("Total number of parameters: {}".format(len(shapes)))

    # Print parameter names
#     enc_dec_param_dict = merge(Selector(encoder).get_parameters(),
#                                Selector(decoder).get_parameters())
#     logger.info("Parameter names: ")
#     for name, value in enc_dec_param_dict.items():
#         logger.info('    {:15}: {}'.format(value.get_value().shape, name))
#     logger.info("Total number of parameters: {}"
#                 .format(len(enc_dec_param_dict)))

    # Set up training model
    logger.info("Building model")
    training_model = Model(cost)
    
    logger.info("Loading parameters from model: {}".format(config['saved_parameters']))
    # load the parameter values from an .npz file
    param_values = LoadNMT.load_parameter_values(config['saved_parameters'])
    LoadNMT.set_model_parameters(training_model, param_values)

    # create the training directory, and copy this config there if directory doesn't exist
#     if not os.path.isdir(config['saveto']):
#         os.makedirs(config['saveto'])
#         shutil.copy(config['config_file'], config['saveto'])

    # Set extensions
    logger.info("Initializing extensions")
    extensions = [
        FinishAfter(after_n_batches=config['finish_after']),
        TrainingDataMonitoring([cost], after_batch=True),
        Printing(after_batch=True),
#         CheckpointNMT(config['saveto'],
#                       every_n_batches=config['save_freq'])
    ]


    # Set up beam search and sampling computation graphs if necessary
    
#     if config['hook_samples'] >= 1 or config['bleu_script'] is not None:
#         logger.info("Building sampling model")
#         sampling_representation = encoder.apply(
#             sampling_input, tensor.ones(sampling_input.shape))
#         # TODO: the generated output actually contains several more values, ipdb to see what they are
#         generated = decoder.generate(sampling_input, sampling_representation)
#         search_model = Model(generated)
#         _, samples = VariableFilter(
#             bricks=[decoder.sequence_generator], name="outputs")(
#                 ComputationGraph(generated[1]))  # generated[1] is next_outputs

    # Add sampling
#     if config['hook_samples'] >= 1:
#         logger.info("Building sampler")
#         extensions.append(
#             Sampler(model=search_model, data_stream=tr_stream,
#                     hook_samples=config['hook_samples'],
#                     every_n_batches=config['sampling_freq'],
#                     src_vocab_size=config['src_vocab_size']))

    # Add early stopping based on bleu
#     if config['bleu_script'] is not None:
#         logger.info("Building bleu validator")
#         extensions.append(
#             BleuValidator(sampling_input, samples=samples, config=config,
#                           model=search_model, data_stream=dev_stream,
#                           normalize=config['normalized_bleu'],
#                           every_n_batches=config['bleu_val_freq']))

    # Reload model if necessary
#     if config['reload']:
#         extensions.append(LoadNMT(config['saveto']))

    # Plot cost in bokeh if necessary
#     if use_bokeh and BOKEH_AVAILABLE:
#         extensions.append(
#             Plot(config['model_save_directory'], channels=[['decoder_cost_cost'], ['validation_set_bleu_score']],
#                  every_n_batches=10))

    # Set up training algorithm
    logger.info("Initializing training algorithm")
    # if there is dropout or random noise, we need to use the output of the modified graph
#     if config['dropout'] < 1.0 or config['weight_noise_ff'] > 0.0:
#         algorithm = GradientDescent(
#             cost=cg.outputs[0], parameters=cg.parameters,
#             step_rule=CompositeRule([StepClipping(config['step_clipping']),
#                                      eval(config['step_rule'])()])
#         )
#     else:
#         algorithm = GradientDescent(
#             cost=cost, parameters=cg.parameters,
#             step_rule=CompositeRule([StepClipping(config['step_clipping']),
#                                      eval(config['step_rule'])()])
#         )

    algorithm = GradientDescent(
        cost=cost, parameters=cg.parameters,
        step_rule=CompositeRule([StepClipping(config['step_clipping']),
                                 eval(config['step_rule'])()],
                               ),
        on_unused_sources='warn'
    )

    # enrich the logged information
    extensions.append(
        Timing(every_n_batches=100)
    )

    # Initialize main loop
    logger.info("Initializing main loop")
    main_loop = MainLoop(
        model=training_model,
        algorithm=algorithm,
        data_stream=tr_stream,
        extensions=extensions
    )

    # Train!
    main_loop.run()

In [ ]:
main(exp_config, masked_stream, dev_stream=None, use_bokeh=False)

INFO:root:Creating theano variables
INFO:root:Building RNN encoder-decoder
INFO:root:Creating computational graph
INFO:root:Building model
INFO:root:Loading parameters from model: /home/chris/projects/neural_mt/archived_models/BERTHA-TEST_Adam_wmt-multimodal_internal_data_dropout0.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/best_bleu_model_1455464992_BLEU31.61.npz
INFO:machine_translation.checkpoint: Loaded to CG (800,)         : /bidirectionalencoder/bidirectionalwmt15/backward.initial_state
INFO:machine_translation.checkpoint: Loaded to CG (1600,)        : /bidirectionalencoder/back_fork/fork_gate_inputs.b
INFO:machine_translation.checkpoint: Loaded to CG (300, 1600)    : /bidirectionalencoder/back_fork/fork_gate_inputs.W
INFO:machine_translation.checkpoint: Loaded to CG (800,)         : /bidirectionalencoder/back_fork/fork_inputs.b
INFO:machine_translation.checkpoint: Loaded to CG (300, 800)     : /bidirectionalencoder/back_fork/fork_inputs.W
INFO:machine_tra


-------------------------------------------------------------------------------
BEFORE FIRST EPOCH
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: False
	 epoch_started: True
	 epochs_done: 0
	 iterations_done: 0
	 received_first_batch: False
	 resumed_from: None
	 training_started: True
Log records from the iteration 0:

sampling_func
num_samples: 5
sampling_func

DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/wrap-xml-modified.perl en /tmp/tmpERNDvO nmt src< /tmp/tmpERNDvO
DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/wrap-xml-modified.perl de /tmp/tmpVq5Fc9 nmt ref< /tmp/tmpkWCZBR
DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/wrap-xml-modified.perl de /tmp/tmpVq5Fc9 nmt tst< /tmp/tmpFrcmfs
DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/mteval-v13a.pl -r /tmp/tmp6fzaWM -s /tmp/tmpVq5Fc9 -t /tmp/tmpW4Qvxy -b -d 2



num_samples: 5
sampling_func

DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/wrap-xml-modified.perl en /tmp/tmpiTJDFk nmt src< /tmp/tmpiTJDFk
DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/wrap-xml-modified.perl de /tmp/tmpCgMfgb nmt ref< /tmp/tmpCsTznu
DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/wrap-xml-modified.perl de /tmp/tmpCgMfgb nmt tst< /tmp/tmpkKGejP
DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/mteval-v13a.pl -r /tmp/tmp9TgT1M -s /tmp/tmpCgMfgb -t /tmp/tmpWUP0zR -b -d 2


...


num_samples: 5

-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: False
	 epoch_started: True
	 epochs_done: 0
	 iterations_done: 5
	 received_first_batch: True
	 resumed_from: None
	 training_started: True
Log records from the iteration 5:
	 decoder_expected_cost_cost: -921.005371094




DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/wrap-xml-modified.perl en /tmp/tmpIUmZxu nmt src< /tmp/tmpIUmZxu
DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/wrap-xml-modified.perl de /tmp/tmpMF_N8d nmt ref< /tmp/tmpFCW3_W
DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/wrap-xml-modified.perl de /tmp/tmpMF_N8d nmt tst< /tmp/tmpoVCzSo
DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/mteval-v13a.pl -r /tmp/tmpP0TCEb -s /tmp/tmpMF_N8d -t /tmp/tmpdVQuv6 -b -d 2


-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: False
	 epoch_started: True
	 epochs_done: 0
	 iterations_done: 6
	 received_first_batch: True
	 resumed_from: None
	 training_started: True
Log records from the iteration 6:
	 decoder_expected_cost_cost: -770.002990723

sampling_func
num_samples: 5
sampling_func

DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/wrap-xml-modified.perl en /tmp/tmphlOHud nmt src< /tmp/tmphlOHud
DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/wrap-xml-modified.perl de /tmp/tmpWKdqWE nmt ref< /tmp/tmpI9onEz
DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/wrap-xml-modified.perl de /tmp/tmpWKdqWE nmt tst< /tmp/tmpGEhnGm
DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/mteval-v13a.pl -r /tmp/tmpC03hqx -s /tmp/tmpWKdqWE -t /tmp/tmpxpF7IQ -b -d 2



num_samples: 5
sampling_func

In [ ]:
masked_stream.get_data()